In [1]:
import pyodbc 
import pandas as pd
import json

# Cognitive Services Text Analytics SDK
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
from configparser import ConfigParser

In [2]:
# Load configuration from config.ini
config = ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
# Retrieve text analytics connection settings from config.ini
access_key = config.get('text-analytics', 'access_key')
endpoint = config.get('text-analytics', 'endpoint')

In [4]:
# Build a connection object to  Connect to SQL Server 
conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=" + config.get('database', 'server') + ";"
                      "Database=" + config.get('database', 'db_name') + ";"
                      "UID=" + config.get('database', 'user_name') + ";"
                      "PWD=" + config.get('database', 'user_password') + ";"
                      "Trusted_Connection=no;")

In [11]:
# Build the query to retrieve documents from the database

# The string below has the curly brackets {} that will be replaced by the batch size using 
# the query.format() when running the query

query = "select top ({}) \
    CONVERT(nvarchar(12), id) AS [id], \
    'en' AS language, \
    comments AS [text] \
    from dbo.reviews \
    where sentiment_score is null \
    order by id asc"

batch_size = 7

In [12]:
# Connect, query, and create a dataframe with the results ( id, language, text )
comments_df = pd.read_sql_query(query.format(batch_size), conn)

In [13]:
# Authenticate Text Analytics Client
credentials = CognitiveServicesCredentials(access_key)
text_analytics = TextAnalyticsClient(endpoint=endpoint, credentials=credentials)

In [14]:
# Converts the dataframe to a json string, then to a list of json docs (one item per row)
documents = json.loads(comments_df.to_json(orient="records"))
# List of JSON docs looks like this: [ {...} , {...} ]

In [15]:
# Call the sentiment cognitive services
response = text_analytics.sentiment(documents=documents)

In [16]:
# Display the results
for document in response.documents:
    print("Document Id: ", document.id, ", Sentiment Score: ",
        "{:.2f}".format(document.score))

Document Id:  18219329 , Sentiment Score:  0.85
Document Id:  18805710 , Sentiment Score:  0.93
Document Id:  19377840 , Sentiment Score:  0.91
Document Id:  19441162 , Sentiment Score:  0.92
Document Id:  19506534 , Sentiment Score:  0.25
